<a href="https://colab.research.google.com/github/tomonari-masada/course2023-nlp/blob/main/12_evaluate_text_similarity_with_language_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 言語モデルを用いたテキストの類似度評価


* 今回は、3種類の言語モデルを使ってテキストをembedする。
* その後、コサイン類似度でテキスト間の類似度を求める。

**ランタイムのタイプはGPUに設定しておく。**

## インストール

* 今回は、軽量なモデルはSentence Transformersライブラリから使うことにする。
  * https://www.sbert.net/
* `fugashi[unidic-lite]`はcl-tohoku/bert-base-japanese-v3を使うために必要。
* `auto-gptq`は量子化されたモデルを使うために必要。
  * https://huggingface.co/blog/gptq-integration

In [ ]:
!pip install fugashi[unidic-lite] sentence-transformers accelerate datasets auto-gptq

In [ ]:
import random
import numpy as np
from tqdm import tqdm_notebook
import torch
import torch.nn.functional as F
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModel
from auto_gptq import AutoGPTQForCausalLM

def set_seed(seed: int):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)

set_seed(123)

## データセット
* 今回は、Hugging Face Hubから、ライブドア・ニュースコーパスを取得する。
  * `random_state`を指定して、ランダムにtrain/validation/test setsへ8:1:1の割合で初めから分割しておく。

In [ ]:
dataset = load_dataset(
    "shunk031/livedoor-news-corpus",
    train_ratio=0.8,
    val_ratio=0.1,
    test_ratio=0.1,
    random_state=42, # 再現性の確保
    shuffle=True,
)

In [ ]:
dataset["train"]["title"][:10]

* 記事のカテゴリをPyTorchのテンソルに変換しておく。

In [ ]:
dataset.keys()

In [ ]:
category = {}
for key in dataset.keys():
  category[key] = torch.tensor(dataset[key]["category"])

In [ ]:
type(category["train"])

## (A) cl-tohoku/bert-base-japanese-v3

* このモデルについては、下記を参照。
  * https://huggingface.co/cl-tohoku/bert-base-japanese-v3

* `sentence_transformers`ライブラリを使って、モデルをダウンロードする。
  * `cl-tohoku/bert-base-japanese-v3`は、embedding向けにfinetuningはされていない。

In [ ]:
model = SentenceTransformer("cl-tohoku/bert-base-japanese-v3")

* 簡単なテキストをembedしてみる。

In [ ]:
corpus = [
    "これはりんごです。",
    "これはりんごですか？",
    "あれはりんごです。",
]

In [ ]:
embeddings = model.encode(corpus, convert_to_tensor=True)

In [ ]:
normalized_embeddings = F.normalize(embeddings, dim=-1)

In [ ]:
normalized_embeddings @ normalized_embeddings.t()

* ライブドア・ニュースコーパスのタイトルをすべてembedする。
  * 15秒ぐらいで終わる。

In [ ]:
embeddings_bert = {}
for key in dataset.keys():
  embeddings_bert[key] = model.encode(dataset[key]["title"], convert_to_tensor=True)

* embeddingの次元を確認する。

In [ ]:
embeddings_bert["train"].shape

* validation setとtraining setとのすべてのペアで類似度を計算する。

In [ ]:
similarities = torch.matmul(
    F.normalize(embeddings_bert["validation"], dim=-1),
    F.normalize(embeddings_bert["train"], dim=-1).t()
)

In [ ]:
similarities.shape

In [ ]:
sorted_indices = torch.argsort(similarities, descending=True).cpu()

In [ ]:
sorted_indices.shape

* validation setのテキスト0について、上位テキストのカテゴリを確認する。

In [ ]:
category["train"][sorted_indices[0,:20]]

In [ ]:
category["validation"][0]

* 最も近いテキストのカテゴリが、正解カテゴリに一致するかで評価してみる。

In [ ]:
(category["train"][sorted_indices[:,0]] == category["validation"]).sum() / len(category["validation"])

## (B) intfloat/multilingual-e5-small
* E5というモデルについては、下記を参照。
  * https://huggingface.co/intfloat/multilingual-e5-base
  * https://hironsan.hatenablog.com/entry/2023/07/05/073150

* このモデルは、テキストのembedding向けにすでにfinetuningされている。

In [ ]:
model = SentenceTransformer("intfloat/multilingual-e5-base")

In [ ]:
normalized_embeddings = F.normalize(model.encode(corpus, convert_to_tensor=True), dim=-1)
normalized_embeddings @ normalized_embeddings.t()

In [ ]:
embeddings_e5 = {}
for key in dataset.keys():
  embeddings_e5[key] = model.encode(dataset[key]["title"], convert_to_tensor=True)

In [ ]:
similarities = torch.matmul(
    F.normalize(embeddings_e5["validation"], dim=-1),
    F.normalize(embeddings_e5["train"], dim=-1).t()
)

In [ ]:
sorted_indices = torch.argsort(similarities, descending=True).cpu()

In [ ]:
print(category["validation"][2], category["train"][sorted_indices[2,:20]])

In [ ]:
(category["train"][sorted_indices[:,0]] == category["validation"]).sum() / len(category["validation"])

## (C) dahara1/weblab-10b-instruction-sft-GPTQ
* https://huggingface.co/dahara1/weblab-10b-instruction-sft-GPTQ

* `matsuo-lab/weblab-10b-instruction-sft`をAutoGPTQで量子化したモデル。
  * https://huggingface.co/matsuo-lab/weblab-10b-instruction-sft

In [ ]:
quantized_model_dir = "dahara1/weblab-10b-instruction-sft-GPTQ"
model_basename = "gptq_model-4bit-128g"

tokenizer = AutoTokenizer.from_pretrained(quantized_model_dir)

model = AutoGPTQForCausalLM.from_quantized(
  quantized_model_dir,
  model_basename=model_basename,
  use_safetensors=True,
  device="cuda:0",
  )

In [ ]:
batch_dict = tokenizer(corpus, padding=True, truncation=True, return_tensors='pt').to("cuda")
with torch.no_grad():
  outputs = model(
    input_ids=batch_dict.input_ids,
    attention_mask=batch_dict.attention_mask,
    output_hidden_states=True,
    )

In [ ]:
def average_pool(last_hidden_states, attention_mask):
  last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
  return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

* modelのoutputの詳細は、下記を参照。
  * https://huggingface.co/docs/transformers/main_classes/output

In [ ]:
outputs.keys()

In [ ]:
embeddings = average_pool(outputs.hidden_states[-1], batch_dict.attention_mask).cpu()

In [ ]:
embeddings = embeddings.type(torch.float32)
normalized_embeddings = F.normalize(embeddings, dim=-1)
normalized_embeddings @ normalized_embeddings.t()

* 11分ぐらいかかる。

In [ ]:
batch_size = 16

embeddings_list = {}
for key in dataset.keys():
  corpus = dataset[key]["title"]
  offset = 0
  embeddings_list[key] = list()
  for offset in tqdm_notebook(range(0, len(corpus), batch_size)):
    batch_dict = tokenizer(
        corpus[offset:offset+batch_size],
        padding=True, truncation=True, return_tensors='pt'
        ).to("cuda")
    with torch.no_grad():
      last_hidden_state = model(
          input_ids=batch_dict.input_ids,
          attention_mask=batch_dict.attention_mask,
          output_hidden_states=True,
          ).hidden_states[-1].cpu()
    tmp_embeddings = average_pool(last_hidden_state, batch_dict.attention_mask.cpu())
    embeddings_list[key].append(tmp_embeddings)
    offset += batch_size

In [ ]:
embeddings_weblab = {}
for key in dataset.keys():
  embeddings_weblab[key] = torch.concat(embeddings_list[key]).type(torch.float32)
  print(key, embeddings_weblab[key].shape)

In [ ]:
for key in dataset.keys():
  torch.save(embeddings_weblab[key], f"livedoor_weblab-10b-instruction-sft-GPTQ_{key}.pt")

In [ ]:
similarities = torch.matmul(
    F.normalize(embeddings_weblab["validation"], dim=-1),
    F.normalize(embeddings_weblab["train"], dim=-1).t()
)

In [ ]:
sorted_indices = torch.argsort(similarities, descending=True).cpu()

In [ ]:
print(category["validation"][0], category["train"][sorted_indices[0,:20]])

In [ ]:
(category["train"][sorted_indices[:,0]] == category["validation"]).sum() / len(category["validation"])

# 課題
* 最低限、上のコードをすべて動かそう。
* 余裕がある人は、validation setのテキストのカテゴリを、上のように1-NNではなく、k>1のk-NNで予測すると予測性能が上がるかどうか、調べてみよう。